In [1]:
import napari
%gui qt5

In [2]:
from brainlit.utils.ngl_pipeline import NeuroglancerSession
from brainlit.viz.swc import *
import numpy as np
from skimage import io

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/python_jsonschema_objects/__init__.py:53: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  self.schema["$schema"]


## Loading entire neuron from AWS 


`napari.components.viewer_model.ViewerModel.add_swc` does this via the following functions in the `napari.layers.swc.swc` module

1. `swc.read_s3` to read the s3 file into a pd.DataFrame
2. `swc.swc_to_voxel` to convert the coordinates from spatial to voxel coordinates
3. `swc.df_to_graph` to convert the DataFrame into a netwrokx.DiGraph
4. `swc.graph_to_paths` to convert from a graph into a list of paths
5. `ViewerModel.add_shapes` to add the paths as a shape layer into the napari viewer

### 1. `read_s3`
This function parses the swc file into a pd.DataFrame. Each row is a vertex in the swc file with the following information: 

`sample number`

`structure identifier`

`x coordinate`

`y coordinate`

`z coordinate`

`radius of dendrite`

`sample number of parent`

The coordinates are given in spatial units of micrometers ([swc specification](http://www.neuronland.org/NLMorphologyConverter/MorphologyFormats/SWC/Spec.html))

In [26]:
s3_path = "s3://mouse-light-viz/precomputed_volumes/brain1_segments"
seg_id = 2
mip = 1
df = read_s3(s3_path, seg_id, mip)
df.head()

Downloading: 100%|██████████| 1/1 [00:00<00:00,  5.75it/s]


,sample,structure,x,y,z,r,parent
0,1,0,4713.0,4470.0,3857.0,1.0,-1
1,4,192,4721.0,4445.0,3849.0,1.0,1
2,7,64,4723.0,4446.0,3851.0,1.0,4
3,8,0,4728.0,4449.0,3852.0,1.0,7
4,14,0,4746.0,4445.0,3858.0,1.0,8


### 2. `swc_to_voxel`

If we want to overlay the swc file with a corresponding image, we need to make sure that they are in the same coordinate space. Because an image in an array of voxels, it makes sense to convert the vertices in the dataframe from spatial units into voxel units.

Given the `spacing` (spatial units/voxel) and `origin` (spatial units) of the image, `swc_to_voxel` does the conversion by using the following equation:

$voxel = \frac{spatial - origin}{spacing}$

In [4]:
spacing = np.array([0.29875923,0.3044159,0.98840415])
origin = np.array([70093.276,15071.596,29306.737])

df_voxel = swc_to_voxel(df=df, spacing=spacing, origin=origin)
df_voxel.head()

,sample,structure,x,y,z,r,parent
0,1,0,-218839,-34826,-25748,1.0,-1
1,4,192,-218813,-34908,-25756,1.0,1
2,7,64,-218806,-34905,-25754,1.0,4
3,8,0,-218789,-34895,-25753,1.0,7
4,14,0,-218729,-34908,-25747,1.0,8


### 3. `df_to_graph`
A neuron is a graph with no cycles (tree). While napari does not support displaying graph objects, it can display multiple paths. 

The DataFrame already contains all the possible edges in the neurons. Each row in the DataFrame is an edge. For example, from the above we can see that `sample 2` has `parent 1`, which represents edge `(1,2)`. `sample 1` having `parent -1` means that `sample 1` is the root of the tree.

`swc.df_to_graph` reads DataFrame and converts it into a networkx directional graph.

In [27]:
G = df_to_graph(df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

Number of nodes: 1650
Number of edges: 1649


Sample 1 coordinates (x,y,z)
4713 4470 3857


### 4. `graph_to_paths`
This function takes in a graph and returns a list of non-overlapping paths. The union of the paths forms the graph.

The algorithm works by:

1. Find longest path in the graph ([networkx.algorithms.dag.dag_longest_path](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.dag.dag_longest_path.html))
2. Remove longest path from graph
3. Repeat steps 1 and 2 until there are no more edges left in the graph

In [28]:
paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")
print(type(paths))

The graph was decomposed into 179 paths
<class 'list'>


In [29]:
paths[0]

array([[4713, 4470, 3857],
       [4721, 4445, 3849],
       [4723, 4435, 3843],
       [4737, 4410, 3838],
       [4763, 4369, 3832],
       [4797, 4345, 3821],
       [4833, 4309, 3812],
       [4844, 4272, 3801],
       [4875, 4238, 3794],
       [4910, 4202, 3790],
       [4914, 4192, 3790],
       [4935, 4156, 3787],
       [4939, 4145, 3786],
       [4956, 4118, 3781],
       [4981, 4073, 3773],
       [4993, 4039, 3766],
       [5006, 3998, 3759],
       [5010, 3981, 3757],
       [5019, 3931, 3749],
       [5033, 3883, 3738],
       [5040, 3862, 3737],
       [5046, 3820, 3729],
       [5043, 3806, 3724],
       [5035, 3816, 3722],
       [5024, 3795, 3719],
       [5006, 3761, 3717],
       [4999, 3754, 3716],
       [5003, 3745, 3711],
       [5023, 3728, 3699],
       [5041, 3693, 3689],
       [5064, 3667, 3674],
       [5099, 3640, 3660],
       [5132, 3621, 3650],
       [5147, 3602, 3643],
       [5180, 3575, 3631],
       [5190, 3528, 3622],
       [5216, 3501, 3614],
 

### 5. `ViewerModel.add_shapes`
napari displays "layers". The most common layer is the image layer. In order to display the neuron, we use `path` from the [shapes](https://napari.org/tutorials/shapes) layer

In [30]:
viewer = napari.Viewer(ndisplay=3)
#viewer.add_image(img)
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', name='Skeleton 2')

<Shapes layer 'Skeleton 2' at 0x16ba38090>

## Loading sub-neuron

The image of the entire brain has dimensions of (33792, 25600, 13312) voxels. G-002 spans a sub-image of (7386, 9932, 5383) voxels. Both are too big to load in napari and overlay the neuron.
To circumvent this, we can crop out a smaller region of the neuron, load the sub-neuron, and load the corresponding sub-image.

In order to get a sub-neuron, we need to specify the `bounding_box` that will be used to crop the neuron. `bounding_box` is a length 2 tuple. The first element is one corner of the bounding box (inclusive) and the second element is the opposite corner of the bounding box (exclusive). Both corners are in voxel units.

`add_swc` can do all of this automatically when given `bounding_box` by following these steps:

1. `read_s3` to read the swc file into a pd.DataFrame
2. `swc_to_voxel` to convert the coordinates from spatial to voxel coordinates
3. `df_to_graph` to convert the DataFrame into a netwrokx.DiGraph
**3.1 `swc.get_sub_neuron` to crop the graph by `bounding_box`**
4. `graph_to_paths` to convert from a graph into a list of paths
5. `ViewerModel.add_shapes` to add the paths as a shape layer into the napari viewer

### `get_sub_neuron`
This function crops a graph by removing edges. It removes edges that do not intersect the bounding box.

Edges that intersect the bounding box will have at least one of its vertices be contained by the bounding box. The algorithm follows this principle by checking the neighborhood of vertices.

For each vertex *v* in the graph:

1. Find vertices belonging to local neighborhood of *v*
2. If vertex *v* or any of its local neighborhood vertices are in the bounding box, do nothing. Otherwise, remove vertex *v* and its edges from the graph

We check the neighborhood of *v* along with *v* because we want the sub-neuron to show all edges that pass through the bounding box, including edges that are only partially contained.

In [31]:
# Create an NGL session to get the bounding box
ngl_sess = NeuroglancerSession(mip = 1)
img, bbbox, vox = ngl_sess.pull_voxel(2, 300, 1, 1, 1)
bbox = bbbox.to_list()
print(bbox)
box = (bbox[:3], bbox[3:])
print(box)

Downloading:   0%|          | 0/3 [00:02<?, ?it/s]


[7524, 2400, 3120, 7722, 2550, 3276]
([7524, 2400, 3120], [7722, 2550, 3276])


In [34]:
G_sub = get_sub_neuron(G, box)
paths_sub = graph_to_paths(G_sub)
viewer = napari.Viewer(ndisplay=3)
viewer.add_shapes(data=paths_sub, shape_type='path', edge_color='blue', name='Skeleton 2')

# overlay corresponding image
image_path = 'G-002_15312-4400-6448_15840-4800-6656.tif'
img_comp = io.imread(image_path)
img_comp = np.swapaxes(img_comp,0,2)
viewer.add_image(img_comp)

<Image layer 'Image' at 0x16fd519d0>